# Valuing European-Style Swaptions Across Models

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.products.rates import *
from financepy.utils import *
from financepy.products.rates.ibor_single_curve import IborSingleCurve
from financepy.market.curves.interpolator import InterpTypes

####################################################################
# FINANCEPY BETA Version 0.200 - This build:  14 Jul 2021 at 16:00 #
# **** NEW PEP8 COMPLIANT VERSION -- PLEASE UPDATE YOUR CODE  **** #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



## Building a Libor Discount Curve

In [3]:
value_date = Date(28, 2, 2014)
settle_date = Date(4, 3, 2014)

We have a vector of dates and zero rates

In [4]:
depoDCCType = DayCountTypes.THIRTY_E_360_ISDA
depos = []

depo = IborDeposit(settle_date, "1W", 0.0023, depoDCCType); depos.append(depo)
depo = IborDeposit(settle_date, "1M", 0.0023, depoDCCType); depos.append(depo)
depo = IborDeposit(settle_date, "3M", 0.0023, depoDCCType); depos.append(depo)
depo = IborDeposit(settle_date, "6M", 0.0023, depoDCCType); depos.append(depo)

# No convexity correction provided so I omit interest rate futures

swaps = []
swapType = SwapTypes.PAY
fixedDCCType = DayCountTypes.ACT_365F
fixed_freq_type = FrequencyTypes.SEMI_ANNUAL

swap = IborSwap(settle_date, "3Y", swapType, 0.00790, fixed_freq_type, fixedDCCType); swaps.append(swap)
swap = IborSwap(settle_date, "4Y", swapType, 0.01200, fixed_freq_type, fixedDCCType); swaps.append(swap)
swap = IborSwap(settle_date, "5Y", swapType, 0.01570, fixed_freq_type, fixedDCCType); swaps.append(swap)
swap = IborSwap(settle_date, "6Y", swapType, 0.01865, fixed_freq_type, fixedDCCType); swaps.append(swap)
swap = IborSwap(settle_date, "7Y", swapType, 0.02160, fixed_freq_type, fixedDCCType); swaps.append(swap)
swap = IborSwap(settle_date, "8Y", swapType, 0.02350, fixed_freq_type, fixedDCCType); swaps.append(swap)
swap = IborSwap(settle_date, "9Y", swapType, 0.02540, fixed_freq_type, fixedDCCType); swaps.append(swap)
swap = IborSwap(settle_date, "10Y", swapType, 0.0273, fixed_freq_type, fixedDCCType); swaps.append(swap)
swap = IborSwap(settle_date, "15Y", swapType, 0.0297, fixed_freq_type, fixedDCCType); swaps.append(swap)
swap = IborSwap(settle_date, "20Y", swapType, 0.0316, fixed_freq_type, fixedDCCType); swaps.append(swap)
swap = IborSwap(settle_date, "25Y", swapType, 0.0335, fixed_freq_type, fixedDCCType); swaps.append(swap)
swap = IborSwap(settle_date, "30Y", swapType, 0.0354, fixed_freq_type, fixedDCCType); swaps.append(swap)

libor_curve = IborSingleCurve(value_date, depos, [], swaps)

In [5]:
exercise_date = settle_date.add_tenor("5Y")
swap_maturity_date = exercise_date.add_tenor("5Y")
swap_fixed_coupon = 0.040852
swap_fixed_frequency_type = FrequencyTypes.SEMI_ANNUAL
swapFixedDayCountType = DayCountTypes.THIRTY_E_360_ISDA
swapFloatFrequencyType = FrequencyTypes.QUARTERLY
swapFloatDayCountType = DayCountTypes.ACT_360
swapNotional = ONE_MILLION
swapType = SwapTypes.PAY  
cal_type = CalendarTypes.WEEKEND
bd_type = BusDayAdjustTypes.NONE
dg_type = DateGenRuleTypes.BACKWARD

In [6]:
swaption = IborSwaption(settle_date,
                            exercise_date,
                            swap_maturity_date,
                            swapType,
                            swap_fixed_coupon,
                            swap_fixed_frequency_type,
                            swapFixedDayCountType, 
                            swapNotional, 
                            swapFloatFrequencyType,
                            swapFloatDayCountType,
                            cal_type, 
                            bd_type,
                            dg_type)

## Valuation using Black's Model

In [7]:
model = Black(0.1533)

In [8]:
swaption.value(settle_date, libor_curve, model)

23177.897388443453

## Valuation using Shifted Black

In [9]:
model = BlackShifted(0.1533, 0.008)

In [10]:
swaption.value(settle_date, libor_curve, model)

27708.384724442658

##  Valuation using SABR

In [11]:
model = SABR(0.132, 0.5, 0.5, 0.5)

In [12]:
swaption.value(settle_date, libor_curve, model)

104569.2906552164

## Valuation using Shifted SABR

In [13]:
model = SABRShifted(0.1, 0.2, 0.1, 0.15, -0.005)

In [14]:
swaption.value(settle_date, libor_curve, model)

163421.0373631705

## Valuation using Hull-White

In [15]:
model = HWTree(0.01, 0.02)

In [16]:
swaption.value(settle_date, libor_curve, model)

34187.751028830324

## Valuation using Black-Karasinski

In [17]:
model = BKTree(0.2, 0.05)

In [18]:
swaption.value(settle_date, libor_curve, model)

23484.270979088065

## Internals

In [19]:
print(swaption)

OBJECT TYPE: IborSwaption
SETTLEMENT DATE: 04-MAR-2014
EXERCISE DATE: 04-MAR-2019
SWAP FIXED LEG TYPE: SwapTypes.PAY
SWAP MATURITY DATE: 04-MAR-2024
SWAP NOTIONAL: 1000000
FIXED COUPON: 4.0852
FIXED FREQUENCY: FrequencyTypes.SEMI_ANNUAL
FIXED DAY COUNT: DayCountTypes.THIRTY_E_360_ISDA
FLOAT FREQUENCY: FrequencyTypes.QUARTERLY
FLOAT DAY COUNT: DayCountTypes.ACT_360
PV01: 4.160171830498574
FWD SWAP RATE: 4.087016466038407
FWD DF TO EXPIRY: 0.923742195746149


We can see that the forward swap rate almost equals the fixed coupon. The underlying swap is close to being ATM forward.

In [20]:
swaption.print_swap_fixed_leg()

START DATE: 04-MAR-2019
MATURITY DATE: 04-MAR-2024
COUPON (%): 4.0852
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT: DayCountTypes.THIRTY_E_360_ISDA
PAY_DATE     ACCR_START   ACCR_END     DAYS  YEARFRAC    RATE      PAYMENT       DF          PV        CUM PV
04-SEP-2019  04-MAR-2019  04-SEP-2019   180  0.500000   4.08520     20426.00  0.90796633     18546.12     18546.12
04-MAR-2020  04-SEP-2019  04-MAR-2020   180  0.500000   4.08520     20426.00  0.89260443     18232.34     36778.46
04-SEP-2020  04-MAR-2020  04-SEP-2020   180  0.500000   4.08520     20426.00  0.87443197     17861.15     54639.61
04-MAR-2021  04-SEP-2020  04-MAR-2021   180  0.500000   4.08520     20426.00  0.85691681     17503.38     72142.99
04-SEP-2021  04-MAR-2021  04-SEP-2021   180  0.500000   4.08520     20426.00  0.84055350     17169.15     89312.13
04-MAR-2022  04-SEP-2021  04-MAR-2022   180  0.500000   4.08520     20426.00  0.82476188     16846.59    106158.72
04-SEP-2022  04-MAR-2022  04-SEP-2022   180  0.50

Copyright (c) 2020 Dominic O'Kane